In [1]:
import os
os.environ["STPSF_PATH"] = "/d/ret1/Taylor/stpsf-data/" 
import webbpsf
os.environ["STPSF_PATH"] = "/d/ret1/Taylor/stpsf-data/" #TJ for some reason this only works if you do this line twice... no idea why

print(os.path.exists(os.environ["STPSF_PATH"]))
print(os.environ["STPSF_PATH"]) #TJ check that this kernel has access to the filter files
import numpy as np
import matplotlib.pyplot as plt
import glob
import re
import sys
from astropy.io import fits
from astropy.visualization import simple_norm, imshow_norm
from ipywidgets import interact, Dropdown
from astropy.wcs import WCS
from astropy.constants import c
from photutils.aperture import CircularAperture, aperture_photometry
import astropy.units as u
from astropy.table import Table
from tabulate import tabulate
from pathlib import Path
from tqdm.notebook import tqdm
from astropy.convolution import convolve_fft, Gaussian2DKernel


#parent_dir = Path().resolve().parent #TJ current notebook's parent directory

os.chdir('/d/ret1/Taylor/jupyter_notebooks/Research') #TJ change working directory to be the parent directory
from Py_files.Basic_analysis import * #import basic functions from custom package
from Py_files.Image_vs_spectra import * 
#TJ import flux calibration functions (mainly compare_IFU_to_image(IFU_filepath, image_filepath, filter_filepath, loc, radius))

def convolve(IFU_fits_file, instrument, filter, output_file = None):
    '''Convolve an IFU cube to the PSF of the provided filter.
    -------------
    
    Parameters
    -------------
    IFU_fits_file : type = str - string to location of IFU fits that you want to convolve.
    instrument : type = str - Either "MIRI" or "NIRCam" depending on wavelength
    filter : type = str - string describing the filter name (case sensitive), for example "F335M"
    output_file (optional, defaults to use the IFU_file with _convolved_to_{filter}) : type = str - name of the convolved file
    
    Returns
    -------------
    Path to newly convolved file as a string
    '''   

    IFU_hdul = fits.open(IFU_fits_file)
    header = IFU_hdul["SCI"].header
    cube = SpectralCube.read(IFU_fits_file, hdu='SCI')
    spectral_axis = cube.spectral_axis  #TJ in meters
    # === Load webbpsf instrument ===
    if instrument == 'NIRCam':
        inst = webbpsf.NIRCam()
    elif instrument == "MIRI":
        inst = webbpsf.MIRI()
    inst.filter = filter

    # === Prepare output cube ===
    convolved_data = np.zeros_like(cube.unmasked_data[:].value)
    tqdm_kwargs = {
        'dynamic_ncols': True,  # Auto-adjusts width
        'mininterval': 0.5,     # Update every 0.5 seconds (optional)
        'position': 0,          # Fix position (set to 0 for notebooks)
        'leave': True           # Leaves progress bar after completion
    }
    # === Loop through wavelengths and convolve ===
    for i, wavelength in enumerate(tqdm(spectral_axis, desc=f"Convolving to {filter}")):
        psf = inst.calc_psf(monochromatic=wavelength.to(u.m).value)    
        psf_data = psf[0].data
        psf_data /= psf_data.sum()  # Normalize PSF to conserve flux
    
        image_slice = cube.filled_data[...].value[i]  # 2D image at this wavelength
        convolved_slice = convolve_fft(image_slice, psf_data, normalize_kernel=True, boundary='fill', fill_value=0)
        convolved_data[i] = convolved_slice
    
    # === Save the convolved cube ===
    out_hdu = fits.PrimaryHDU(convolved_data, header=header)
    if output_file:
        out_hdu.writeto(f"Data_files/IFU_files/{output_file}", overwrite=True)
        print(f"✅ PSF convolution complete and saved as {output_file}")
        return f"Data_files/IFU_files/{output_file}"
    else:
        out_hdu.writeto(f"Data_files/IFU_files/{IFU_fits_file}_convolved_to_{filter}.fits", overwrite=True)
        print(f"✅ PSF convolution complete and saved as {IFU_fits_file}_convolved_to_{filter}")
        return f"Data_files/IFU_files/{IFU_fits_file}_convolved_to_{filter}.fits"

        
    


/tmp/ipykernel_93963/230208821.py:3: DeprecationWarning: "
        This message is for information only and WebbPSF will continue to function as normal.
        The WebbPSF library has been moved/renamed to STPSF.
        Please see https://stpsf.readthedocs.io/en/stable/ for more information.
        WebbPSF is now an alias of STPSF and is running code from the STPSF library.
        
  import webbpsf


True
/d/ret1/Taylor/stpsf-data/


In [2]:
# === User Inputs ===

if __name__ == "__main__":
    #TJ add the path to psf data files if needed (may need to go back through adding the path in terminal)
    #filters = ['F335M', 'F360M', 'F405N', 'F430M', 'F444W']
    filters = ['F335M', 'F360M', 'F405N', 'F430M', 'F444W', 'F560W', 'F770W', 'F1000W', 'F1130W', 'F1280W', 'F1500W', 'F1800W', 'F2100W'] 
    # === Set up ===
    
    for filter in filters:
        if filter in ['F335M', 'F360M', 'F405N', 'F430M', 'F444W']:
            IFU_filepath = "Data_files/IFU_files/M51_SW_f290lp_g395m-f290lp_s3d.fits"
            instrument = 'NIRCam'  # or 'MIRI' if using MIRI cube
        if filter in ['F560W']:
            IFU_filepath = 'Data_files/IFU_files/SW_IFU_ch1-shortmediumlong_s3d.fits'
            instrument = "MIRI"
        if filter in ['F770W', 'F1000W']:
            IFU_filepath = 'Data_files/IFU_files/SW_IFU_ch2-shortmediumlong_s3d.fits'
            instrument = "MIRI"
        if filter in ['F1130W', 'F1280W', 'F1500W']:
            IFU_filepath = 'Data_files/IFU_files/SW_IFU_ch3-shortmediumlong_s3d.fits'
            instrument = "MIRI"
        if filter in ['F1800W', 'F2100W']:
            IFU_filepath = 'Data_files/IFU_files/SW_IFU_ch4-shortmediumlong_s3d.fits'
            instrument = "MIRI"
            
        IFU_hdul = fits.open(IFU_filepath)
        header = IFU_hdul["SCI"].header
        cube = SpectralCube.read(IFU_filepath, hdu='SCI')
        wcs = WCS(header)
        spectral_axis = cube.spectral_axis  #TJ in meters
        # === Load webbpsf instrument ===
        if instrument == 'NIRCam':
            inst = webbpsf.NIRCam()
        elif instrument == "MIRI":
            inst = webbpsf.MIRI()
        inst.filter = filter
    
        # === Prepare output cube ===
        convolved_data = np.zeros_like(cube.unmasked_data[:].value)
        tqdm_kwargs = {
            'dynamic_ncols': True,  # Auto-adjusts width
            'mininterval': 0.5,     # Update every 0.5 seconds (optional)
            'position': 0,          # Fix position (set to 0 for notebooks)
            'leave': True           # Leaves progress bar after completion
        }
        # === Loop through wavelengths and convolve ===
        for i, wavelength in enumerate(tqdm(spectral_axis, desc=f"Convolving to {filter}")):
            psf = inst.calc_psf(monochromatic=wavelength.to(u.m).value)    
            psf_data = psf[0].data
            psf_data /= psf_data.sum()  # Normalize PSF to conserve flux
        
            image_slice = cube.filled_data[...].value[i]  # 2D image at this wavelength
            convolved_slice = convolve_fft(image_slice, psf_data, normalize_kernel=True, boundary='fill', fill_value=0)
            convolved_data[i] = convolved_slice
        
        # === Save the convolved cube ===
        out_hdu = fits.PrimaryHDU(convolved_data, header=header)
        out_hdu.writeto(f"Data_files/IFU_files/M51_IFU_convolved_to_{inst.filter}.fits", overwrite=True)
        
        print("✅ PSF convolution complete and saved as 'M51_IFU_convolved.fits'")


Processing:   0%|          | 0/1341 [00:00<?, ?it/s]

✅ PSF convolution complete and saved as 'M51_IFU_convolved.fits'


Processing:   0%|          | 0/1341 [00:00<?, ?it/s]

✅ PSF convolution complete and saved as 'M51_IFU_convolved.fits'


Processing:   0%|          | 0/1341 [00:00<?, ?it/s]

...

✅ PSF convolution complete and saved as 'M51_IFU_convolved.fits'


Processing:   0%|          | 0/1834 [00:00<?, ?it/s]

✅ PSF convolution complete and saved as 'M51_IFU_convolved.fits'


Processing:   0%|          | 0/1834 [00:00<?, ?it/s]

✅ PSF convolution complete and saved as 'M51_IFU_convolved.fits'


<Quantity [2.87089489, 2.87268489, 2.87447489, ..., 5.2659149 , 5.2677049 ,
           5.2694949 ] um>

'F2100W'

['F115W',
 'F140M',
 'F150W',
 'F164N',
 'F182M',
 'F187N',
 'F200W',
 'F210M',
 'F212N',
 'F250M',
 'F300M',
 'F335M',
 'F360M',
 'F405N',
 'F430M',
 'F444W',
 'F560W',
 'F770W',
 'F1000W',
 'F1130W',
 'F1280W',
 'F1500W',
 'F1800W',
 'F2100W']